In [8]:
from langchain.chains import TransformChain
import requests

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")


Found Azure OpenAI API Base Endpoint: https://swed-oai.openai.azure.com/


In [2]:
bing_search_url = "https://api.bing.microsoft.com/v7.0/search"
bing_subscription_key = os.getenv("BING_SUBSCRIPTION_KEY")

In [3]:
# Open the file
with open('guest_name.txt', 'r') as f:
    guest = f.read()

# Print the content
print(guest)

Neil deGrasse Tyson


In [9]:
def bing_grounding(input_dict:dict) -> dict:
    print("Calling Bing Search API to get bio for guest...\n")
    search_term = input_dict["guest"]
    print("Search term is " + search_term)

    headers = {"Ocp-Apim-Subscription-Key": bing_subscription_key}
    params = {"q": search_term, "textDecorations": True, "textFormat": "HTML"}
    response = requests.get(bing_search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()
    #print(search_results)

    # Parse out a bio.  
    bio = search_results["webPages"]["value"][0]["snippet"]
    
    print("Bio:\n")
    print(bio)
    print("\n")

    return {"bio": bio}


In [10]:
bing_chain = TransformChain(input_variables=["guest"], output_variables=["bio"], transform=bing_grounding)
bio = bing_chain.run(guest)


Calling Bing Search API to get bio for guest...

Search term is Neil deGrasse Tyson
Bio:

<b><b>Neil deGrasse Tyson</b></b> (US: / d ə ˈ ɡ r æ s / də-GRASS or UK: / d ə ˈ ɡ r ɑː s / də-GRAHSS; born October 5, 1958) is an American astrophysicist, author, and science communicator<b>. <b>Tyso</b>n</b> studied at Harvard University , the University of Texas at Austin , and Columbia University .




In [15]:
# Specify the file path
file_path = "bio.txt"

# Write the content to the file
with open(file_path, "w") as file:
    file.write(bio)

# Function calling in Assistance API


In [17]:
import os
from dotenv import load_dotenv
import json
from openai import AzureOpenAI

# load environment variables
load_dotenv()

True

In [ ]:

import requests
import time


from pathlib import Path
from typing import Optional

In [14]:
client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = "2023-12-01-preview"
)
model =  os.getenv("AZURE_OPENAI_CHAT_MODEL")
print(model)

gpt-4


## Define function to perform Bing search

In [15]:
def search_bing(query):
    """
    Perform a bing search against the given query

    @param query: Search query
    @return: List of search results

    """
    headers = {"Ocp-Apim-Subscription-Key": bing_subscription_key}
    params = {"q": query, "textDecorations": False}
    response = requests.get(bing_search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()

    output = []

    for result in search_results["webPages"]["value"]:
        output.append({"title": result["name"], "link": result["url"], "snippet": result["snippet"]})

    return json.dumps(output)
    

## Test Bing search and evaluate the search results

In [18]:
search_bing(guest)

'[{"title": "Neil deGrasse Tyson - Wikipedia", "link": "https://en.wikipedia.org/wiki/Neil_deGrasse_Tyson", "snippet": "Neil deGrasse Tyson (US: / d \\u0259 \\u02c8 \\u0261 r \\u00e6 s / d\\u0259-GRASS or UK: / d \\u0259 \\u02c8 \\u0261 r \\u0251\\u02d0 s / d\\u0259-GRAHSS; born October 5, 1958) is an American astrophysicist, author, and science communicator. Tyson studied at Harvard University , the University of Texas at Austin , and Columbia University ."}, {"title": "Neil deGrasse Tyson - Wikipedia", "link": "https://nl.wikipedia.org/wiki/Neil_deGrasse_Tyson", "snippet": "Neil deGrasse Tyson (New York, 5 oktober 1958) is een Amerikaanse astrofysicus en wetenschapscommunicator. Hij houdt zich onder meer bezig met stervorming, exploderende sterren, dwergstelsels en de structuur van de Melkweg."}, {"title": "Neil deGrasse Tyson", "link": "https://neildegrassetyson.com/", "snippet": "\\u00a9 2024 Neil deGrasse Tyson"}, {"title": "Neil deGrasse Tyson | Biography, Books, TV Shows, & Fact

## Create a prompt for GPT model

In [41]:
messages = [{"role": "user", "content": "Provide a very short biography of "+guest + " including lates events"}]

## Test GPT model response without grounding

In [42]:
gpt_response = client.chat.completions.create(
            model=model,
            messages=messages,
        )  # get a new response from the model where it can see the function response
    
bio = gpt_response.choices[0].message.content
print(bio)

Neil deGrasse Tyson is a prominent American astrophysicist, cosmologist, author, and science communicator. Born on October 5, 1958, in Manhattan, New York, Tyson developed an early interest in astronomy and decided to dedicate his life to the field after visiting the Hayden Planetarium at a young age. Tyson earned a BA in Physics from Harvard University before completing his PhD in Astrophysics at Columbia University.

He has served as the Frederick P. Rose Director of the Hayden Planetarium at the Rose Center for Earth and Space in New York City since 1996. Tyson is well-known for hosting the educational science television show "NOVA ScienceNow" on PBS from 2006 to 2011 and more recently, the television series "Cosmos: A Spacetime Odyssey", a sequel to the late Carl Sagan's renowned series "Cosmos: A Personal Voyage".

Tyson is a prolific author, writing several books on astronomy and space, with the most recent being "Cosmic Queries: StarTalk’s Guide to Who We Are, How We Got Here, a

## Define Tools for GPT model, that include Bing search function

In [37]:
#messages = [{"role": "user", "content": "Provide the latest information about "+guest}]

tools = [
    {
        "type": "function",
        "function": {
            "name": "search_bing",
            "description": "Searches bing to get up-to-date information about people",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query",
                    }
                },
                "required": ["query"],
            },
        },
    }
]

## Send request to GPT model and see if there any tools calling

In [38]:
response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
        )

response_message = response.choices[0].message
tool_calls = response_message.tool_calls
print(response_message)
print(tool_calls)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Gq2UMNrOQe1uqmMTZhO98YcB', function=Function(arguments='{\n  "query": "Neil deGrasse Tyson biography"\n}', name='search_bing'), type='function')])
[ChatCompletionMessageToolCall(id='call_Gq2UMNrOQe1uqmMTZhO98YcB', function=Function(arguments='{\n  "query": "Neil deGrasse Tyson biography"\n}', name='search_bing'), type='function')]


## Go through function calling and extract messages to see Bing search results

In [43]:
if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "search_bing": search_bing,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                query=function_args.get("query")
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
print(messages)

[{'role': 'user', 'content': 'Provide a very short biography of Neil deGrasse Tyson including lates events'}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Gq2UMNrOQe1uqmMTZhO98YcB', function=Function(arguments='{\n  "query": "Neil deGrasse Tyson biography"\n}', name='search_bing'), type='function')]), {'tool_call_id': 'call_Gq2UMNrOQe1uqmMTZhO98YcB', 'role': 'tool', 'name': 'search_bing', 'content': '[{"title": "Neil deGrasse Tyson - Wikipedia", "link": "https://en.wikipedia.org/wiki/Neil_deGrasse_Tyson", "snippet": "Neil deGrasse Tyson (US: / d \\u0259 \\u02c8 \\u0261 r \\u00e6 s / d\\u0259-GRASS or UK: / d \\u0259 \\u02c8 \\u0261 r \\u0251\\u02d0 s / d\\u0259-GRAHSS; born October 5, 1958) is an American astrophysicist, author, and science communicator. Tyson studied at Harvard University , the University of Texas at Austin , and Columbia University ."}, {"title": "Neil deGrasse Tyson: Biography, Astrophys

## Ask GPT model to generate short bio of the person based on search results

In [44]:
second_response = client.chat.completions.create(
            model=model,
            messages=messages,
        )  # get a new response from the model where it can see the function response
    
bio = second_response.choices[0].message.content
print(bio)

Neil deGrasse Tyson is an American astrophysicist, author, and science communicator. He was born on October 5, 1958. Tyson studied at Harvard University, the University of Texas at Austin, and Columbia University. He is renowned for popularizing science with his books and regular appearances on radio and television. Tyson’s television hosting roles have included TV series such as Origins and NOVA scienceNOW, as well as the radio show StarTalk. 

Tyson is the fifth head since 1935 of the world-renowned Hayden Planetarium in New York City, occupying its Frederick P. Rose Directorship and also a research associate of the Department of Astrophysics at the American Museum of Natural History. Some of his notable books include “The Pluto Files: The Rise and Fall of America’s Favorite Planet”.

One of the latest appearances of Tyson was in the television show "Cosmos: Possible Worlds" in 2020.


### Compare with output created without grounding
Neil deGrasse Tyson is a prominent American astrophysicist, cosmologist, author, and science communicator. Born on October 5, 1958, in Manhattan, New York, Tyson developed an early interest in astronomy and decided to dedicate his life to the field after visiting the Hayden Planetarium at a young age. Tyson earned a BA in Physics from Harvard University before completing his PhD in Astrophysics at Columbia University.

He has served as the Frederick P. Rose Director of the Hayden Planetarium at the Rose Center for Earth and Space in New York City since 1996. Tyson is well-known for hosting the educational science television show "NOVA ScienceNow" on PBS from 2006 to 2011 and more recently, the television series "Cosmos: A Spacetime Odyssey", a sequel to the late Carl Sagan's renowned series "Cosmos: A Personal Voyage".

Tyson is a prolific author, writing several books on astronomy and space, with the most recent being "Cosmic Queries: StarTalk’s Guide to Who We Are, How We Got Here, and Where We're Going", published in 2021. He is also the host of the podcast "StarTalk", where he discusses various aspects of science with famous guests.

In the latest events, Tyson remains an active advocate for science, contributing regularly to television, radio, and social media platforms to simplify complex scientific concepts for public understanding.

## Save the biography in file

In [45]:
# Specify the file path
file_path = "bio.txt"

# Write the content to the file
with open(file_path, "w") as file:
    file.write(bio)